# Data Cleaning

### WiFi Log Data

In [1]:
import pandas as pd
# Import package numpy for numeric computing
import numpy as np
# Import package matplotlib for visualisation/plotting
import matplotlib.pyplot as plt

# Reading from a csv file, into a data frame
wifi_data = pd.read_csv("wifi_data.csv")

# Show first 5 rows of data frame
wifi_data.head()


,Unnamed: 0,Room,Timestamp,Assoc,Auth
0,21,B003,1447101189,0,0
1,22,B003,1447101491,0,0
2,23,B003,1447101786,0,0
3,24,B003,1447102095,0,0
4,25,B003,1447102392,0,0


In [2]:
wifi_data.head()

,Unnamed: 0,Room,Timestamp,Assoc,Auth
0,21,B003,1447101189,0,0
1,22,B003,1447101491,0,0
2,23,B003,1447101786,0,0
3,24,B003,1447102095,0,0
4,25,B003,1447102392,0,0


In [3]:
#dropping first column Unamed:0 as we don't need that column in our analysis
wifi_data.drop(wifi_data.columns[[0]], axis=1, inplace=True)
wifi_data.head()

,Room,Timestamp,Assoc,Auth
0,B003,1447101189,0,0
1,B003,1447101491,0,0
2,B003,1447101786,0,0
3,B003,1447102095,0,0
4,B003,1447102392,0,0


In [4]:
wifi_data.shape

(12417, 4)

In [5]:
#checking the number of data with NaN
wifi_data.isnull().sum()

Room         0
Timestamp    0
Assoc        0
Auth         0
dtype: int64

In [6]:
#checking for duplicates
wifi_data.duplicated().sum()

0

#### Split column Timestamp data into Weekday, Date and Time columns

In [7]:

from datetime import datetime
format_timestamp = lambda x:datetime.fromtimestamp(int(x))

wifi_data['Timestamp'] = wifi_data['Timestamp'].map(format_timestamp)

In [8]:
wifi_data.head()

,Room,Timestamp,Assoc,Auth
0,B003,2015-11-09 20:33:09,0,0
1,B003,2015-11-09 20:38:11,0,0
2,B003,2015-11-09 20:43:06,0,0
3,B003,2015-11-09 20:48:15,0,0
4,B003,2015-11-09 20:53:12,0,0


In [9]:
#iterate through all rows and split column time for three different values
weekdays, dates, times = [], [], []
for timestamp in wifi_data['Timestamp']:
    weekdays.append(timestamp.weekday())
    dates.append(timestamp.strftime("%Y-%m-%d"))
    times.append(timestamp.strftime("%H:%M:%S"))

In [10]:
wifi_data['Weekday'] = weekdays
wifi_data['Date'] = dates
wifi_data['Time'] = times

In [11]:
#Mapping weekday number to name
import calendar
format_weekday = lambda x:calendar.day_name[x]

wifi_data['Weekday'] = wifi_data['Weekday'].map(format_weekday)

In [12]:
wifi_data.head()

,Room,Timestamp,Assoc,Auth,Weekday,Date,Time
0,B003,2015-11-09 20:33:09,0,0,Monday,2015-11-09,20:33:09
1,B003,2015-11-09 20:38:11,0,0,Monday,2015-11-09,20:38:11
2,B003,2015-11-09 20:43:06,0,0,Monday,2015-11-09,20:43:06
3,B003,2015-11-09 20:48:15,0,0,Monday,2015-11-09,20:48:15
4,B003,2015-11-09 20:53:12,0,0,Monday,2015-11-09,20:53:12


#### Removing all data from Saturday and Sunday

In [13]:
wifi_data = wifi_data[(wifi_data['Weekday'] != 'Saturday') & (wifi_data['Weekday'] != 'Sunday')]

#### Filtering data between time 9:00 and 17:00 of each day

In [14]:
wifi_data = wifi_data.set_index('Timestamp')
wifi_data = wifi_data.between_time('09:00','17:00')

#### Selecting max value of Associated for each column

In [15]:
hours = ["{}:00".format(hour) for hour in range(9, 18)]  #we get list: ['9:00', '10:00', ..., '16:00']

In [16]:
output_data = pd.DataFrame(columns=wifi_data.columns)  #creating empty output table based on input table

for room in wifi_data['Room'].unique():  # for each room
    room_data = wifi_data[wifi_data['Room'] == room]  
    for date in room_data['Date'].unique():  # for each date
        date_wifi_data = room_data[room_data['Date'] == date]  
        
        for index in range(len(hours)-1):  # for each range of hour
            time_data = date_wifi_data.between_time(hours[index], hours[index+1])  
            if time_data["Assoc"].count():  
                max_value = time_data["Assoc"].max()  # choosing the max Assoc from selected data
                max_time_data = time_data[time_data["Assoc"] == max_value].iloc[0]  
                output_data = output_data.append(max_time_data)  

In [17]:
#changing hour to format in range 9:00 - 10:00, 10:00-11:00 etc
def format_timerange(x):
    hour = int(x.split(':')[0])  
    return "{}:00-{}:00".format(str(hour).zfill(2), str(hour+1).zfill(2))  # zfill adding extra zero to the hour
 
output_data['Time'] = output_data ['Time'].map(format_timerange)

In [18]:
output_data.head(25)

,Room,Assoc,Auth,Weekday,Date,Time
2015-11-13 09:42:13,B002,9,9,Friday,2015-11-13,09:00-10:00
2015-11-13 10:54:19,B002,17,17,Friday,2015-11-13,10:00-11:00
2015-11-13 11:05:01,B002,19,19,Friday,2015-11-13,11:00-12:00
2015-11-13 12:40:15,B002,20,20,Friday,2015-11-13,12:00-13:00
2015-11-13 13:35:58,B002,16,16,Friday,2015-11-13,13:00-14:00
2015-11-13 14:16:18,B002,20,19,Friday,2015-11-13,14:00-15:00
2015-11-13 15:07:01,B002,21,21,Friday,2015-11-13,15:00-16:00
2015-11-13 16:07:19,B002,11,11,Friday,2015-11-13,16:00-17:00
2015-11-03 09:58:34,B002,21,21,Tuesday,2015-11-03,09:00-10:00
2015-11-03 10:54:35,B002,32,32,Tuesday,2015-11-03,10:00-11:00


In [19]:
#putting table header in order
output_data = output_data [['Room', 'Weekday', 'Date', 'Time', 'Assoc', 'Auth']]
output_data.head()

,Room,Weekday,Date,Time,Assoc,Auth
2015-11-13 09:42:13,B002,Friday,2015-11-13,09:00-10:00,9,9
2015-11-13 10:54:19,B002,Friday,2015-11-13,10:00-11:00,17,17
2015-11-13 11:05:01,B002,Friday,2015-11-13,11:00-12:00,19,19
2015-11-13 12:40:15,B002,Friday,2015-11-13,12:00-13:00,20,20
2015-11-13 13:35:58,B002,Friday,2015-11-13,13:00-14:00,16,16


In [20]:
output_data.shape

(249, 6)

### Reading  from a csv file, into a data frame - TIMETABLE 

In [21]:

# Reading from a csv file, into a data frame
timetable = pd.read_csv("timetable.csv")
# Show first 5 rows of data frame
timetable.head()

,Unnamed: 0,room_id,timestamp,code
0,0,B002,1446454800,COMP30190
1,1,B002,1446458400,COMP40660
2,2,B002,1446462000,not available
3,3,B002,1446465600,COMP30250
4,4,B002,1446469200,COMP41690


In [22]:
timetable.dtypes

Unnamed: 0     int64
room_id       object
timestamp      int64
code          object
dtype: object

In [23]:
#dropping first column Unamed:0 as we don't need that column in our analysis
timetable.drop(timetable.columns[[0]], axis=1, inplace=True)
timetable.head()

,room_id,timestamp,code
0,B002,1446454800,COMP30190
1,B002,1446458400,COMP40660
2,B002,1446462000,not available
3,B002,1446465600,COMP30250
4,B002,1446469200,COMP41690


In [24]:
#checking the number of data with NaN
#NaN means that there is no class at that time
timetable.isnull().sum()

room_id      0
timestamp    0
code         0
dtype: int64

In [25]:
#replace string 'not avaialbe' to 'empty'
timetable.code.replace(['not available'], ['empty'], inplace=True)

In [26]:
timetable.head()

,room_id,timestamp,code
0,B002,1446454800,COMP30190
1,B002,1446458400,COMP40660
2,B002,1446462000,empty
3,B002,1446465600,COMP30250
4,B002,1446469200,COMP41690


In [27]:
timetable.duplicated().sum()

0

#### Split column timestamp into Weekday, Date and Time columns

In [28]:
timetable['timestamp'] = timetable['timestamp'].map(format_timestamp)

In [29]:
#iterate through all rows and split column timestamp for three different values
weekdays, dates, times = [], [], []
for timestamp in timetable['timestamp']:
    weekdays.append(timestamp.weekday())
    dates.append(timestamp.strftime("%Y-%m-%d"))
    times.append(timestamp.strftime("%H:%M:%S"))

In [30]:
timetable['Weekday'] = weekdays
timetable['Date'] = dates
timetable['Time'] = times

In [31]:
#Mapping weekday number to name
timetable['Weekday'] = timetable['Weekday'].map(format_weekday)

In [32]:
timetable.head()

,room_id,timestamp,code,Weekday,Date,Time
0,B002,2015-11-02 09:00:00,COMP30190,Monday,2015-11-02,09:00:00
1,B002,2015-11-02 10:00:00,COMP40660,Monday,2015-11-02,10:00:00
2,B002,2015-11-02 11:00:00,empty,Monday,2015-11-02,11:00:00
3,B002,2015-11-02 12:00:00,COMP30250,Monday,2015-11-02,12:00:00
4,B002,2015-11-02 13:00:00,COMP41690,Monday,2015-11-02,13:00:00


In [33]:
timetable['Time'] = timetable['Time'].map(format_timerange)

In [34]:
timetable.head()

,room_id,timestamp,code,Weekday,Date,Time
0,B002,2015-11-02 09:00:00,COMP30190,Monday,2015-11-02,09:00-10:00
1,B002,2015-11-02 10:00:00,COMP40660,Monday,2015-11-02,10:00-11:00
2,B002,2015-11-02 11:00:00,empty,Monday,2015-11-02,11:00-12:00
3,B002,2015-11-02 12:00:00,COMP30250,Monday,2015-11-02,12:00-13:00
4,B002,2015-11-02 13:00:00,COMP41690,Monday,2015-11-02,13:00-14:00


In [35]:
#renaming column name room_id to Room and code to Module
timetable = timetable.rename(columns = {'room_id':'Room','code':'Module'})
timetable.head()

,Room,timestamp,Module,Weekday,Date,Time
0,B002,2015-11-02 09:00:00,COMP30190,Monday,2015-11-02,09:00-10:00
1,B002,2015-11-02 10:00:00,COMP40660,Monday,2015-11-02,10:00-11:00
2,B002,2015-11-02 11:00:00,empty,Monday,2015-11-02,11:00-12:00
3,B002,2015-11-02 12:00:00,COMP30250,Monday,2015-11-02,12:00-13:00
4,B002,2015-11-02 13:00:00,COMP41690,Monday,2015-11-02,13:00-14:00


In [36]:
timetable.shape

(271, 6)

### Reading  from a csv file, into a data frame -  Ground Truth data

In [37]:
# Reading from a csv file, into a data frame
ground = pd.read_csv("ground.csv")
# Show first 5 rows of data frame
ground.head()

,Room,over_3,perc_occupied,survey_id,timestamp
0,B004,1,25,0,1446454800
1,B004,1,50,1,1446458400
2,B004,1,25,2,1446462000
3,B004,1,25,3,1446465600
4,B004,1,50,4,1446469200


In [38]:
ground.dtypes


Room             object
over_3            int64
perc_occupied     int64
survey_id         int64
timestamp         int64
dtype: object

In [39]:
ground.duplicated().sum()

0

In [40]:
#checking the number of data with Nan
ground.isnull().sum()

Room             0
over_3           0
perc_occupied    0
survey_id        0
timestamp        0
dtype: int64

#### Split timestamp data into Weekday, Date and Time columns

In [41]:
#changing unix timestampt to date format
ground['timestamp'] = pd.to_datetime(ground['timestamp'],unit='s')
ground.head()

,Room,over_3,perc_occupied,survey_id,timestamp
0,B004,1,25,0,2015-11-02 09:00:00
1,B004,1,50,1,2015-11-02 10:00:00
2,B004,1,25,2,2015-11-02 11:00:00
3,B004,1,25,3,2015-11-02 12:00:00
4,B004,1,50,4,2015-11-02 13:00:00


In [42]:
#iterate through all rows and split column timestamp for three different values
weekdays, dates, times = [], [], []
for timestamp in ground['timestamp']:
    weekdays.append(timestamp.weekday())
    dates.append(timestamp.strftime("%Y-%m-%d"))
    times.append(timestamp.strftime("%H:%M:%S"))

In [43]:
ground['Weekday'] = weekdays
ground['Date'] = dates
ground['Time'] = times

In [44]:
#Mapping weekday number to name

ground['Weekday'] = ground['Weekday'].map(format_weekday)

In [45]:
#changing hour to format in range 9:00 - 10:00, 10:00-11:00 etc
ground['Time'] = ground['Time'].map(format_timerange)

In [46]:
ground.head()

,Room,over_3,perc_occupied,survey_id,timestamp,Weekday,Date,Time
0,B004,1,25,0,2015-11-02 09:00:00,Monday,2015-11-02,09:00-10:00
1,B004,1,50,1,2015-11-02 10:00:00,Monday,2015-11-02,10:00-11:00
2,B004,1,25,2,2015-11-02 11:00:00,Monday,2015-11-02,11:00-12:00
3,B004,1,25,3,2015-11-02 12:00:00,Monday,2015-11-02,12:00-13:00
4,B004,1,50,4,2015-11-02 13:00:00,Monday,2015-11-02,13:00-14:00


In [47]:
ground.shape

(480, 8)

In [48]:
ground.dtypes

Room                     object
over_3                    int64
perc_occupied             int64
survey_id                 int64
timestamp        datetime64[ns]
Weekday                  object
Date                     object
Time                     object
dtype: object

#### Reading from a csv file, into a data frame - Room

In [49]:
# Reading from a csv file, into a data frame
room = pd.read_csv("room.csv")
# Show first 5 rows of data frame
room.head()

,Room,Capacity,Room_Type
0,B004,160,Lecture Theatre
1,B002,90,Classroom
2,B003,90,Classroom
3,B106,90,A.L.E
4,B108,40,Seminar


In [50]:
room.duplicated().sum()

0

In [51]:
#checking the number of data with Nan
room.isnull().sum()

Room         0
Capacity     0
Room_Type    0
dtype: int64

In [52]:
room.shape

(6, 3)

In [53]:
room.dtypes

Room         object
Capacity      int64
Room_Type    object
dtype: object

#### Reading from a csv file, into a data frame - Modules

In [54]:
# Reading from a csv file, into a data frame
modules = pd.read_csv("modules.csv")
# Show first 5 rows of data frame
modules.head()

,students,code
0,53,COMP40660
1,60,COMP41690
2,22,COMP30250
3,45,COMP30110
4,27,COMP40370


In [55]:
#renaming column name room_id to Room and code to Module
modules = modules.rename(columns = {'code':'Module'})
modules.head()

,students,Module
0,53,COMP40660
1,60,COMP41690
2,22,COMP30250
3,45,COMP30110
4,27,COMP40370


In [56]:
modules.duplicated().sum()

0

In [57]:
modules.isnull().sum()

students    0
Module      0
dtype: int64

In [58]:
modules.dtypes

students     int64
Module      object
dtype: object

In [59]:
modules.shape

(34, 2)

#### Merging table room with table timetable on the same column name - room 

In [60]:
result = timetable.merge(room,on=['Room'])

In [61]:
result.head()

,Room,timestamp,Module,Weekday,Date,Time,Capacity,Room_Type
0,B002,2015-11-02 09:00:00,COMP30190,Monday,2015-11-02,09:00-10:00,90,Classroom
1,B002,2015-11-02 10:00:00,COMP40660,Monday,2015-11-02,10:00-11:00,90,Classroom
2,B002,2015-11-02 11:00:00,empty,Monday,2015-11-02,11:00-12:00,90,Classroom
3,B002,2015-11-02 12:00:00,COMP30250,Monday,2015-11-02,12:00-13:00,90,Classroom
4,B002,2015-11-02 13:00:00,COMP41690,Monday,2015-11-02,13:00-14:00,90,Classroom


In [62]:
result.shape

(271, 8)

#### Merging table result1 with table modules on the same columns name - Module

In [63]:
result1 = pd.merge(result, modules, on=['Module'], how='outer')

In [64]:
result1.shape

(271, 9)

In [65]:
result1.head()

,Room,timestamp,Module,Weekday,Date,Time,Capacity,Room_Type,students
0,B002,2015-11-02 09:00:00,COMP30190,Monday,2015-11-02,09:00-10:00,90,Classroom,29
1,B002,2015-11-04 09:00:00,COMP30190,Wednesday,2015-11-04,09:00-10:00,90,Classroom,29
2,B002,2015-11-06 11:00:00,COMP30190,Friday,2015-11-06,11:00-12:00,90,Classroom,29
3,B002,2015-11-06 12:00:00,COMP30190,Friday,2015-11-06,12:00-13:00,90,Classroom,29
4,B002,2015-11-09 09:00:00,COMP30190,Monday,2015-11-09,09:00-10:00,90,Classroom,29


#### Merging table output_data with groune and result tables on the same columns name - Room, Date, Time 

In [66]:
result2 = output_data.merge(ground,on=['Room','Weekday','Date','Time']).merge(result1,on=['Room','Weekday','Date','Time'])

In [67]:
result2.shape

(218, 15)

In [68]:
result2.head()

,Room,Weekday,Date,Time,Assoc,Auth,over_3,perc_occupied,survey_id,timestamp_x,timestamp_y,Module,Capacity,Room_Type,students
0,B002,Friday,2015-11-13,09:00-10:00,9,9,0,0,152,2015-11-13 09:00:00,2015-11-13 09:00:00,COMP30220,90,Classroom,38
1,B002,Friday,2015-11-13,10:00-11:00,17,17,0,0,153,2015-11-13 10:00:00,2015-11-13 10:00:00,empty,90,Classroom,NaN
2,B002,Friday,2015-11-13,11:00-12:00,19,19,0,0,154,2015-11-13 11:00:00,2015-11-13 11:00:00,COMP30190,90,Classroom,29
3,B002,Friday,2015-11-13,12:00-13:00,20,20,0,0,155,2015-11-13 12:00:00,2015-11-13 12:00:00,COMP30190,90,Classroom,29
4,B002,Friday,2015-11-13,13:00-14:00,16,16,0,0,156,2015-11-13 13:00:00,2015-11-13 13:00:00,COMP30170,90,Classroom,51


In [69]:

#dropping first column Unamed:0
result2.drop(result2.columns[[8,9,10]], axis=1, inplace=True)
result2.head()

,Room,Weekday,Date,Time,Assoc,Auth,over_3,perc_occupied,Module,Capacity,Room_Type,students
0,B002,Friday,2015-11-13,09:00-10:00,9,9,0,0,COMP30220,90,Classroom,38
1,B002,Friday,2015-11-13,10:00-11:00,17,17,0,0,empty,90,Classroom,NaN
2,B002,Friday,2015-11-13,11:00-12:00,19,19,0,0,COMP30190,90,Classroom,29
3,B002,Friday,2015-11-13,12:00-13:00,20,20,0,0,COMP30190,90,Classroom,29
4,B002,Friday,2015-11-13,13:00-14:00,16,16,0,0,COMP30170,90,Classroom,51


In [70]:
result2.shape

(218, 12)

In [71]:
#checking the number of missing data
result2.isnull().sum()


Room              0
Weekday           0
Date              0
Time              0
Assoc             0
Auth              0
over_3            0
perc_occupied     0
Module            0
Capacity          0
Room_Type         0
students         59
dtype: int64

In [73]:
result2.dtypes

Room              object
Weekday           object
Date              object
Time              object
Assoc            float64
Auth             float64
over_3             int64
perc_occupied      int64
Module            object
Capacity           int64
Room_Type         object
students         float64
dtype: object

In [75]:
#Select all columns of type 'object'
object_columns = result2.select_dtypes(['object']).columns
object_columns

Index(['Room', 'Weekday', 'Date', 'Time', 'Module', 'Room_Type'], dtype='object')

In [76]:
#Convert selected columns to type 'category'
for column in object_columns:
    result2[column] = result2[column].astype('category')
result2.dtypes

Room             category
Weekday          category
Date             category
Time             category
Assoc             float64
Auth              float64
over_3              int64
perc_occupied       int64
Module           category
Capacity            int64
Room_Type        category
students          float64
dtype: object

In [78]:
#  Prepare a table with descriptive statistics for all the categorical features.
categorical_features = result2.select_dtypes(['category']).describe().T
categorical_features

,count,unique,top,freq
Room,218,3,B004,76
Weekday,218,5,Wednesday,50
Date,218,9,2015-11-11,25
Time,218,8,09:00-10:00,31
Module,218,34,empty,59
Room_Type,218,2,Classroom,142


In [80]:
# Prepare a table with descriptive statistics for all the continuous features.
# Descriptive stats for all the continuous features.
continuous_features = result2.describe().T
continuous_features

,count,mean,std,min,25%,50%,75%,max
Assoc,218,41.729358,34.189566,0,19,33,58.0,232
Auth,218,41.518349,34.036469,0,19,33,57.5,230
over_3,218,0.697248,0.460506,0,0,1,1.0,1
perc_occupied,218,25.802752,22.564247,0,0,25,50.0,100
Capacity,218,114.403670,33.434197,90,90,90,160.0,160
students,159,58.698113,42.014276,0,27,53,98.0,139


In [ ]:
#column student has 59 NAN value 
#this means that there is no modules during that time so there is no students registered to that modules
#We can fill in any NAN
#In the example below NAN was changed to number 0

df = result2.fillna(0)
df.head()

In [ ]:
df.isnull().sum()

In [ ]:
df.shape

In [ ]:
#checking if there is any row duplicated in dataframe
df.duplicated().sum()

In [ ]:
df.head()

In [ ]:
#renaming columns name
df = df.rename(columns = {'over_3':'Over_3', 'perc_occupied':'Target', 'Assoc':'Associated','Auth': 'Authenticated','students':'Students'})


In [ ]:
df.head()

In [ ]:
#putting columns in order
df = df [['Weekday', 'Date', 'Time', 'Room', 'Room_Type', 'Capacity', 'Module','Students','Over_3', 'Associated','Authenticated', 'Target']]
df.head()

In [ ]:
#saving clean table. That table can be used for data alaytics
df.to_csv('table_ABT.csv')